# SPY Spreadsheet

In [1]:
# use future imports for python 3.x forward compatibility
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

# other imports
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import itable

# project imports
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [5]:
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2019-07-30,301.17,299.49,299.91,300.72,45849000.00,300.72
2019-07-31,301.20,295.20,300.99,297.43,104245200.00,297.43
2019-08-01,300.87,293.96,297.60,294.84,142646600.00,294.84
2019-08-02,294.12,290.90,293.85,292.62,115917700.00,292.62
2019-08-05,288.21,281.73,288.09,284.05,157051184.00,284.05


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [6]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [7]:
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
1993-01-29,26.73,26.59,26.73,26.71,1003200.00,26.71
1993-02-01,26.90,26.73,26.73,26.90,480500.00,26.90
1993-02-02,26.97,26.82,26.88,26.95,201300.00,26.95
1993-02-03,27.26,26.97,26.99,27.24,529400.00,27.24
1993-02-04,27.41,27.03,27.33,27.35,531500.00,27.35


Add technical indicator: 200 day MA

In [8]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [9]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2019-07-30,301.17,299.49,299.91,300.72,45849000.00,300.72,276.23
2019-07-31,301.20,295.20,300.99,297.43,104245200.00,297.43,276.38
2019-08-01,300.87,293.96,297.60,294.84,142646600.00,294.84,276.49
2019-08-02,294.12,290.90,293.85,292.62,115917700.00,292.62,276.60
2019-08-05,288.21,281.73,288.09,284.05,157051184.00,284.05,276.64


Add technical indicator: ATR

In [10]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [11]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr
date,,,,,,,,
2019-07-30,301.17,299.49,299.91,300.72,45849000.00,300.72,276.23,2.16
2019-07-31,301.20,295.20,300.99,297.43,104245200.00,297.43,276.38,2.44
2019-08-01,300.87,293.96,297.60,294.84,142646600.00,294.84,276.49,2.75
2019-08-02,294.12,290.90,293.85,292.62,115917700.00,292.62,276.60,2.84
2019-08-05,288.21,281.73,288.09,284.05,157051184.00,284.05,276.64,3.41


Add technical indicator: 5 day high, and 5 day low

In [12]:
high5 = pd.Series(ts.high).rolling(window=5).max()
low5 = pd.Series(ts.low).rolling(window=5).min()
ts['high5'] = high5
ts['low5'] = low5

In [13]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2019-07-30,301.17,299.49,299.91,300.72,45849000.00,300.72,276.23,2.16,302.23,299.09
2019-07-31,301.20,295.20,300.99,297.43,104245200.00,297.43,276.38,2.44,302.23,295.20
2019-08-01,300.87,293.96,297.60,294.84,142646600.00,294.84,276.49,2.75,302.23,293.96
2019-08-02,294.12,290.90,293.85,292.62,115917700.00,292.62,276.60,2.84,302.01,290.90
2019-08-05,288.21,281.73,288.09,284.05,157051184.00,284.05,276.64,3.41,301.20,281.73


Add technical indicator: RSI, and 2-period cumulative RSI

In [14]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.Series(ts.rsi2).rolling(window=2).sum()
ts['c2rsi2'] = c2rsi2

In [15]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2019-07-30,301.17,299.49,299.91,300.72,45849000.00,300.72,276.23,2.16,302.23,299.09,35.70,94.44
2019-07-31,301.20,295.20,300.99,297.43,104245200.00,297.43,276.38,2.44,302.23,295.20,7.95,43.65
2019-08-01,300.87,293.96,297.60,294.84,142646600.00,294.84,276.49,2.75,302.23,293.96,3.58,11.53
2019-08-02,294.12,290.90,293.85,292.62,115917700.00,292.62,276.60,2.84,302.01,290.90,1.84,5.42
2019-08-05,288.21,281.73,288.09,284.05,157051184.00,284.05,276.64,3.41,301.20,281.73,0.39,2.23


Add technical indicator: Midpoint

In [16]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [17]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2019-07-30,301.17,299.49,299.91,300.72,45849000.00,300.72,276.23,2.16,302.23,299.09,35.70,94.44,300.33
2019-07-31,301.20,295.20,300.99,297.43,104245200.00,297.43,276.38,2.44,302.23,295.20,7.95,43.65,298.20
2019-08-01,300.87,293.96,297.60,294.84,142646600.00,294.84,276.49,2.75,302.23,293.96,3.58,11.53,297.41
2019-08-02,294.12,290.90,293.85,292.62,115917700.00,292.62,276.60,2.84,302.01,290.90,1.84,5.42,292.51
2019-08-05,288.21,281.73,288.09,284.05,157051184.00,284.05,276.64,3.41,301.20,281.73,0.39,2.23,284.97


Add technical indicator: SMA10 of midpoint

In [18]:
sma10mp = pd.Series(ts.mp).rolling(window=10).mean()
ts['sma10mp'] = sma10mp

In [19]:
ts.head(10)

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
1993-01-29,26.73,26.59,26.73,26.71,1003200.00,26.71,nan,nan,nan,nan,nan,nan,26.66,nan
1993-02-01,26.90,26.73,26.73,26.90,480500.00,26.90,nan,nan,nan,nan,nan,nan,26.81,nan
1993-02-02,26.97,26.82,26.88,26.95,201300.00,26.95,nan,nan,nan,nan,100.00,nan,26.90,nan
1993-02-03,27.26,26.97,26.99,27.24,529400.00,27.24,nan,nan,nan,nan,100.00,200.00,27.12,nan
1993-02-04,27.41,27.03,27.33,27.35,531500.00,27.35,nan,nan,27.41,26.59,100.00,200.00,27.22,nan
1993-02-05,27.39,27.18,27.33,27.33,492100.00,27.33,nan,nan,27.41,26.73,89.34,189.34,27.29,nan
1993-02-08,27.43,27.30,27.33,27.33,596100.00,27.33,nan,nan,27.43,26.82,89.34,178.68,27.36,nan
1993-02-09,27.24,27.09,27.24,27.14,122100.00,27.14,nan,nan,27.43,26.97,16.96,106.30,27.16,nan
1993-02-10,27.20,27.07,27.14,27.18,379600.00,27.18,nan,nan,27.43,27.03,37.28,54.24,27.13,nan


Add technical indicator: Standard Deviation

In [20]:
sd = pd.Series(ts.mp).rolling(window=10).std()
ts['sd'] = sd

In [21]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd
date,,,,,,,,,,,,,,,
2019-07-30,301.17,299.49,299.91,300.72,45849000.00,300.72,276.23,2.16,302.23,299.09,35.70,94.44,300.33,299.57,1.33
2019-07-31,301.20,295.20,300.99,297.43,104245200.00,297.43,276.38,2.44,302.23,295.20,7.95,43.65,298.20,299.51,1.38
2019-08-01,300.87,293.96,297.60,294.84,142646600.00,294.84,276.49,2.75,302.23,293.96,3.58,11.53,297.41,299.45,1.46
2019-08-02,294.12,290.90,293.85,292.62,115917700.00,292.62,276.60,2.84,302.01,290.90,1.84,5.42,292.51,298.85,2.64
2019-08-05,288.21,281.73,288.09,284.05,157051184.00,284.05,276.64,3.41,301.20,281.73,0.39,2.23,284.97,297.57,5.14


In [22]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [23]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-07-30,301.17,299.49,299.91,300.72,45849000.00,300.72,276.23,2.16,302.23,299.09,35.70,94.44,300.33,299.57,1.33,302.22,296.92
2019-07-31,301.20,295.20,300.99,297.43,104245200.00,297.43,276.38,2.44,302.23,295.20,7.95,43.65,298.20,299.51,1.38,302.27,296.75
2019-08-01,300.87,293.96,297.60,294.84,142646600.00,294.84,276.49,2.75,302.23,293.96,3.58,11.53,297.41,299.45,1.46,302.37,296.54
2019-08-02,294.12,290.90,293.85,292.62,115917700.00,292.62,276.60,2.84,302.01,290.90,1.84,5.42,292.51,298.85,2.64,304.14,293.57
2019-08-05,288.21,281.73,288.09,284.05,157051184.00,284.05,276.64,3.41,301.20,281.73,0.39,2.23,284.97,297.57,5.14,307.86,287.29


Select a smaller time from for use with itable

In [24]:
df = ts['2018-01-01':]

In [25]:
df.head()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2018-01-02,261.37,260.00,260.43,261.33,86655700.00,261.33,239.06,1.42,261.37,259.26,79.59,103.56,260.69,260.26,0.35,260.96,259.56
2018-01-03,263.15,261.52,261.52,262.99,90070400.00,262.99,239.24,1.45,263.15,259.26,91.14,170.73,262.33,260.40,0.73,261.87,258.94
2018-01-04,264.63,263.05,263.69,264.09,80636400.00,264.09,239.44,1.46,264.63,259.26,94.96,186.09,263.84,260.75,1.31,263.37,258.13
2018-01-05,265.99,264.42,264.97,265.85,83524000.00,265.85,239.65,1.50,265.99,259.26,97.87,192.83,265.21,261.26,1.89,265.05,257.47
2018-01-08,266.51,265.43,265.75,266.34,57319200.00,266.34,239.86,1.47,266.51,260.00,98.39,196.26,265.97,261.81,2.38,266.56,257.06


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [26]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme='theme1'), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [27]:
pt

,HIGH,LOW,OPEN,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,UPPER,LOWER
2018/01/02,261.37,260.00,260.43,261.33,86655700,261.33,239.06,1.42,261.37,259.26,79.59,103.56,260.69,260.26,0.35,260.96,259.56
2018/01/03,263.15,261.52,261.52,262.99,90070400,262.99,239.24,1.45,263.15,259.26,91.14,170.73,262.33,260.40,0.73,261.87,258.94
2018/01/04,264.63,263.05,263.69,264.09,80636400,264.09,239.44,1.46,264.63,259.26,94.96,186.09,263.84,260.75,1.31,263.37,258.13
2018/01/05,265.99,264.42,264.97,265.85,83524000,265.85,239.65,1.50,265.99,259.26,97.87,192.83,265.21,261.26,1.89,265.05,257.47
2018/01/08,266.51,265.43,265.75,266.34,57319200,266.34,239.86,1.47,266.51,260.00,98.39,196.26,265.97,261.81,2.38,266.56,257.06
2018/01/09,267.63,266.50,266.81,266.94,57254000,266.94,240.08,1.45,267.63,261.52,98.99,197.38,267.06,262.53,2.78,268.09,256.98
2018/01/10,266.83,265.37,266.11,266.53,69574300,266.53,240.29,1.46,267.63,263.05,65.63,164.63,266.10,263.16,2.80,268.76,257.57
2018/01/11,268.48,266.96,267.15,268.48,62361500,268.48,240.50,1.50,268.48,264.42,91.84,157.47,267.72,263.94,2.89,269.72,258.16
2018/01/12,270.41,268.44,268.77,270.23,90816100,270.23,240.73,1.53,270.41,265.37,96.56,188.39,269.43,264.85,3.04,270.94,258.76
2018/01/16,272.34,268.54,271.62,269.31,106555100,269.31,240.94,1.69,272.34,265.37,59.95,156.51,270.44,265.88,3.03,271.93,259.83
